In [4]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
import numpy as np
import requests
import datetime
from datetime import datetime, timedelta, date
from dateutil.parser import parse
import matplotlib.pyplot as plt

#Alpaca API imports
import alpaca_trade_api as tradeapi

#Twitter API imports
import tweepy as tw

# NLP & Sentiment imports
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

from dotenv import load_dotenv
load_dotenv()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Kris/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
############################################################
"""
    Authenticates the Alpaca API and Twitter
    Returns a pass/fail statement
"""
############################################################  
    
# Setting twitter access and api keys
bearer_token = os.getenv("TWITTER_BEARER_TOKEN")
consumer_key= os.getenv("TWITTER_API_KEY")
consumer_secret= os.getenv("TWITTER_SECRET_KEY")
access_token= os.getenv("TWITTER_ACCESS_TOKEN")
access_token_secret= os.getenv("TWITTER_ACCESS_TOKEN_SECRET")

# authentication for twitter
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
twitter_api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# test authentication
try:
    twitter_api.verify_credentials()
    auth = "Twitter Authentication Verified"
except:
    auth = "Error During Twitter Authentication"
    
print(auth)

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca_api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

Twitter Authentication Verified


In [18]:
############################################################
"""
    Function that pulls stock data from a given ticker and timeframe.
"""
############################################################

def daily_returns(ticker, timeframe):
        
    # Set current date and the date from one month ago using the ISO format
    current_date = pd.Timestamp.now(tz="America/New_York").isoformat()
    past_date = pd.Timestamp("2021-01-01 00:00", tz="America/New_York").isoformat()

    # Get 4 weeks worth of historical data for AAPL
    df = alpaca_api.get_barset(
        ticker,
        timeframe,
        limit=None,
        start=past_date,
        end=current_date,
        after=None,
        until=None,
    ).df
        
    return df


############################################################
"""
    get_ticker_sentiment(ticker):
    Takes 1 argument, ticker to search tweets for.
    
    Scrapes Twitter for given search words.
    Calculates compound sentiment with VADER sentiment analyzer 
    Normalizes VADER compound score
    Returns Average Daily Sentiment Dataframe with Columns: 
"""
############################################################

def get_ticker_returns(ticker, timeframe):
    
    # alpaca api variables
    ticker = ticker
    timeframe = timeframe

    # call the alpaca api and return a dataframe of daily returns
    daily_df = daily_returns(ticker, timeframe)

    # Drop Outer Table Level and drop extra columns
    daily_df = daily_df.droplevel(axis=1, level=0)
    daily_df = daily_df.drop(columns=["open", "high", "low", "volume"])

    # Get the percent change of the closing prices, drop any NA rows, and reset the index
    # daily_df["percent_change"] = daily_df.pct_change().dropna()
    returns_df = daily_df.pct_change().dropna()

    # Grouping the returns by Hour and taking their average Hourly sentiment
    avg_returns = returns_df.groupby(pd.Grouper(level=0, freq='H')).mean().dropna()
    
    return avg_returns


############################################################
"""
    Twitter: Scrape Tweets and Analyze Sentiment
"""
############################################################

def twitter_sentiment(search_words, date_since, items):
       
    # Initialize the VADER sentiment analyzer
    analyzer = SentimentIntensityAnalyzer()

    # initializing the tweets dataframe
    df = []
    
    # adding retweet filter to search words
    search_words = search_words + " -filter:retweets"
    
    # Fetch top tweets/hastags for given ticker
    tweets = tw.Cursor(twitter_api.search,
              q=search_words,
              lang="en",
              since=date_since
                    ).items(items)
    
    for tweet in tweets:
    
        #storing tweet text
        tweet_fetched = tweet.text 

        # Get date of tweet
        tweet_date = pd.Timestamp(tweet.created_at, tz="America/New_York").isoformat()
        
        try:
            sentiment = analyzer.polarity_scores(tweet_fetched)
            compound = sentiment["compound"]
            #pos = sentiment["pos"]
            #neu = sentiment["neu"]
            #neg = sentiment["neg"]
        
            df.append({
                "date": tweet_date,
                "tweet": tweet_fetched,
                "compound": compound,
                #"positive": pos,
                #"negative": neg,
                #"neutral": neu
            
            })
        
        except AttributeError:
            pass
    
    df = pd.DataFrame(df)
    
    return df

############################################################
"""
    get_twitter_sentiment(search_words):
    Takes 2 argument, word(s) to search tweets for, and number of items (tweets) to return.
    
    Scrapes Twitter for given search words in tweet
    Calculates compound sentiment with VADER sentiment analyzer on each tweet
    Calculates average compound sentiment score each 1 hour
    Normalizes average hourly VADER compound score
    Returns Average Hourly Sentiment Dataframe with Columns: 
"""
############################################################

def get_avg_twitter_sentiment(search_word, ticker):

    # tweepy variables
    #search_words = search_words
    date_since = "2021-01-01"
    items = 1000
    twitter_search_phrase = search_word + " " + ticker

    # call the twitter sentiment function and return a dataframe
    tweets_df = twitter_sentiment(twitter_search_phrase, date_since, items)

    # Changes the date column to proper datetime format
    tweets_df['date'] = pd.to_datetime(tweets_df['date'])
    
    # Grouping the tweets by Hour and taking their average Hourly sentiment
    avg_hourly_sentiment = tweets_df.groupby(pd.Grouper(key='date', freq='H')).mean().dropna()
    
    # Get the normalized sentiment score of -1, 0, 1
    # avg_hourly_sentiment["normalized"] = avg_hourly_sentiment["compound"].apply(lambda x : get_normalized(x))
    
    return avg_hourly_sentiment


############################################################
"""
    Sentiment calculation based on compound score
"""
############################################################

def get_normalized(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.04:  # Positive
        result = 1
    elif score <= -0.04:  # Negative
        result = -1

    return result

############################################################
"""
    combine_sentiment_with_close(avg_hourly_returns, avg_hourly_sentiment):
    Takes 2 arguments: the two dataframes to combine
"""
############################################################

def combine_sentiment_with_close(avg_hourly_sentiment, avg_hourly_returns):

    # Combines the average hourly Twitter sentiment dataframe with the hourly percent change dataframe
    combined_df = avg_hourly_returns.join(avg_hourly_sentiment).dropna()
    
    return combined_df


############################################################
    """
    This function accepts the column number for the features (X) .
    It chunks the data up with a rolling window of Xt - window to predict Xt.
    It returns a numpy array of X.
    
    `df`: The original DataFrame with the time series data.
    `window`: The window size in days of previous closing prices that will be used for the prediction.
    `feature_col_number`: The column number from the original DataFrame where the features are located.
  
    """
############################################################

def window_data(df, window, feature_col_number, target_col_number):

    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

############################################################
    """
    Creating the features (X) data using the window_data() function.
    """
############################################################

def create_features(combined_df):

    window_size = 1
    X, y = window_data(combined_df, window_size, 0, 1)

    # Use the MinMaxScaler to scale data between 0 and 1.
    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    scaler.fit(y)
    y = scaler.transform(y)

    # Reshape the features for the model
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    return X


############################################################
"""
    Machine Learning Model
    Input the Sentiment data and return predicted stock returns
"""
############################################################

def get_predictions(X):
    
    # Load the sentiment model 
    from tensorflow.keras.models import model_from_json

    # load json and create model
    file_path = Path("ml_model/sentiment_model.json")
    with open(file_path, "r") as json_file:
        sentiment_model_json = json_file.read()
    sentiment_loaded_model = model_from_json(sentiment_model_json)
    
    # load weights into new model
    file_path = "ml_model/sentiment_model.h5"
    sentiment_loaded_model.load_weights(file_path)
    
    # Make some predictions with the loaded model
    predicted = sentiment_loaded_model.predict(X)
    
    # Recover the original prices instead of the scaled version
    predicted_prices = scaler.inverse_transform(predicted)
    
    # Create a DataFrame of Predicted values
    predicted_returns = pd.DataFrame({
        "Predicted": predicted_prices.ravel()
        })

    return predicted_returns


############################################################
"""
    main_function:
    Defines the main function.
    1 argument: search word to search twitter and stock prices.
"""
############################################################

def main_function(twitter_search_word, ticker):
    
    timeframe = "15Min"  
    
    twitter_sentiment = get_avg_twitter_sentiment(twitter_search_word, ticker)
    percent_change = get_ticker_returns(ticker, timeframe)
    combined_df = combine_sentiment_with_close(twitter_sentiment, percent_change)
    ml_features = create_features(combined_df)
    predicted_returns = get_predictions(ml_features)
    
    return combined_df

In [19]:
twitter_search_word = "microsoft" #search_word
ticker = "MSFT" #ticker

combined_df = main_function(twitter_search_word, ticker)
combined_df

ValueError: in user code:

    /opt/anaconda3/envs/nlpenv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /opt/anaconda3/envs/nlpenv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/envs/nlpenv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/envs/nlpenv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/envs/nlpenv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/envs/nlpenv/lib/python3.7

In [25]:
predicted_returns.plot(figsize=(20,10));